**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°:1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.



In [41]:
# Pre procesamiento y web scrapping
import requests
from bs4 import BeautifulSoup
import pandas as pd

# modelado
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk

### Lectura de DF

Descargamos los dataset desde el link, para no lidiar con inconvenientes al abrir y cerrar sesiones de colab.

In [ ]:
!wget -O bgg_database.csv 'https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq'
!wget -O IMDB-Movie-Data.csv 'https://drive.google.com/uc?id=1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c'

--2024-11-01 11:57:14--  https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.google.com (drive.google.com)... 173.194.217.113, 173.194.217.102, 173.194.217.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq [following]
--2024-11-01 11:57:14--  https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.107.132, 2607:f8b0:400c:c32::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.107.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1833113 (1.7M) [application/octet-stream]
Saving to: ‘bgg_database.csv’

bgg_database.csv    100%[===================>]   1.75M  --.-KB/s    in 0.04s   

2024-11-01 11:57:18 (47

In [ ]:
df_juegos: pd.DataFrame = pd.read_csv('bgg_database.csv')
df_peliculas: pd.DataFrame = pd.read_csv('IMDB-Movie-Data.csv')

### Exploración y procesado de df_juegos

Tomar criterios de evaluación para ver si borrar columnas o no.

In [ ]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              1000 non-null   int64  
 1   game_name         1000 non-null   object 
 2   game_href         1000 non-null   object 
 3   geek_rating       1000 non-null   float64
 4   avg_rating        1000 non-null   float64
 5   num_voters        1000 non-null   float64
 6   description       1000 non-null   object 
 7   yearpublished     1000 non-null   int64  
 8   minplayers        1000 non-null   int64  
 9   maxplayers        1000 non-null   int64  
 10  minplaytime       1000 non-null   int64  
 11  maxplaytime       1000 non-null   int64  
 12  minage            1000 non-null   int64  
 13  avgweight         1000 non-null   float64
 14  best_num_players  1000 non-null   object 
 15  designers         1000 non-null   object 
 16  mechanics         1000 non-null   object 
 

Podemos visualizar que las siguientes columnas presentan datos que neceistamos trabajar:
- best_num_players: presenta una lista que contiene un diccionario.
- designers, mechanics, categories: son listas.
- description: tiene caracteres '\n'

In [ ]:
df_juegos.sample(5)

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
894,895,Celestia,https://boardgamegeek.com/boardgame/175117/cel...,6.682,6.94,11605.0,"In Celestia, a revamped version of Cloud 9, yo...",2015,2,6,30,30,8,1.3272,"[{'min': 5, 'max': 6}]",['Aaron Weissblum'],"['Betting and Bluffing', 'Dice Rolling', 'Hidd...","['Adventure', 'Aviation / Flight', 'Card Game'..."
256,257,Deception: Murder in Hong Kong,https://boardgamegeek.com/boardgame/156129/dec...,7.245,7.44,22849.0,Deception: Murder in Hong Kong is a game of de...,2014,4,12,20,20,14,1.5789,"[{'min': 6, 'max': 8}]",['Tobey Ho'],"['Communication Limits', 'Deduction', 'Events'...","['Bluffing', 'Deduction', 'Murder/Mystery', 'P..."
263,264,Hallertau,https://boardgamegeek.com/boardgame/300322/hal...,7.235,7.88,5916.0,"The Hallertau in Bavaria, Germany is the large...",2020,1,4,50,140,12,3.2939,"[{'min': 2, 'max': 2}]",['Uwe Rosenberg'],"['Action Points', 'Advantage Token', 'Automati...","['Animals', 'Economic', 'Farming', 'Industry /..."
743,744,Unmatched Game System,https://boardgamegeek.com/boardgame/295564/unm...,6.772,8.24,2007.0,\n\n\n \n GAME SYSTEM\n\n \n \...,2019,2,4,20,40,9,1.9825,"[{'min': 2, 'max': 2}]","['Rob Daviau', 'Justin D. Jacobson']","['Action Points', 'Card Play Conflict Resoluti...","['Card Game', 'Fantasy', 'Fighting', 'Game Sys..."
405,406,Abyss,https://boardgamegeek.com/boardgame/155987/abyss,7.065,7.33,15081.0,"The Abyss power is once again vacant, so the t...",2014,2,4,30,60,14,2.3216,"[{'min': 4, 'max': 4}]","['Bruno Cathala', 'Charles Chevallier']","['Auction/Bidding', 'Hand Management', 'Memory...","['Card Game', 'Fantasy', 'Mythology', 'Nautica..."


In [ ]:
df_juegos.describe()

,rank,geek_rating,avg_rating,num_voters,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,500.500000,7.053187,7.570630,13615.758000,2008.427000,1.760000,4.937000,59.55700,93.397000,11.636000,2.680625
std,288.819436,0.351866,0.391122,15876.211424,135.395031,0.689112,6.272509,43.50627,83.159816,2.252683,0.819375
min,1.000000,6.615000,6.760000,1364.000000,-2200.000000,1.000000,1.000000,0.00000,0.000000,0.000000,1.000000
25%,250.750000,6.770000,7.277500,4608.250000,2011.000000,1.000000,4.000000,30.00000,45.000000,10.000000,2.100300
50%,500.500000,6.970000,7.530000,7983.500000,2016.000000,2.000000,4.000000,60.00000,77.500000,12.000000,2.642300
75%,750.250000,7.256250,7.802500,16097.750000,2019.000000,2.000000,5.000000,60.00000,120.000000,14.000000,3.266025
max,1000.000000,8.415000,9.040000,128735.000000,2024.000000,6.000000,100.000000,480.00000,1200.000000,18.000000,4.818200


In [ ]:
df_juegos.isna().sum() #no encontramos nulos.

,0
rank,0
game_name,0
game_href,0
geek_rating,0
avg_rating,0
num_voters,0
description,0
yearpublished,0
minplayers,0
maxplayers,0


### Exploración y procesado de df_peliculas

In [ ]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  1000 non-null   float64
 11  Metascore           1000 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB


In [ ]:
df_peliculas.sample(5)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
96,97,Kimi no na wa,"Animation,Drama,Fantasy",Two strangers find themselves linked in a biza...,Makoto Shinkai,"Ryûnosuke Kamiki, Mone Kamishiraishi, Ryô Nari...",2016,106,8.6,34110,4.68,79
454,455,I Am Number Four,"Action,Adventure,Sci-Fi",Aliens and their Guardians are hiding on Earth...,D.J. Caruso,"Alex Pettyfer, Timothy Olyphant, Dianna Agron,...",2011,109,6.1,202682,55.09,36
844,845,Unstoppable,"Action,Thriller","With an unmanned, half-mile-long freight train...",Tony Scott,"Denzel Washington, Chris Pine, Rosario Dawson,...",2010,98,6.8,157499,81.56,69
528,529,Godzilla,"Action,Adventure,Sci-Fi",The world is beset by the appearance of monstr...,Gareth Edwards,"Aaron Taylor-Johnson, Elizabeth Olsen, Bryan C...",2014,123,6.4,318058,200.66,62
881,882,Saving Mr. Banks,"Biography,Comedy,Drama",Author P.L. Travers reflects on her childhood ...,John Lee Hancock,"Emma Thompson, Tom Hanks, Annie Rose Buckley, ...",2013,125,7.5,125693,83.30,65


In [ ]:
df_peliculas.isna().sum() # no encontramos nulos.

,0
Rank,0
Title,0
Genre,0
Description,0
Director,0
Actors,0
Year,0
Runtime (Minutes),0
Rating,0
Votes,0



## Web scrapping y procesado: df_libros


In [ ]:
url: str = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'

In [ ]:
response: requests.models = requests.get(url)

# extraigo el contenido de la página web
soup: BeautifulSoup = BeautifulSoup(response.text, 'html.parser')

# luego de la inspección, vemos que necesitamos la información de page_content
libro_div = soup.find('div', class_='page_content')

# divido para quedarme con los títulos
paragraphs = libro_div.find_all('a')

In [ ]:
# creo lista para poder crear el df
libro: list = [par.text for par in paragraphs]
libro = libro[1::] # mediante slicing salteo la primera linea con información que no nos interesa.

In [ ]:
df_libros: pd.DataFrame = pd.DataFrame(libro, columns=['Libro'])

In [ ]:
def limpieza_top(libro: str) -> bool:
  """
  Chequea si en string existe el substring 'Top 1000'. Retorna un booleano
  """
  if 'Top 1000' in libro:
    return True
  else:
    return False

In [ ]:
df_libros['es_ruido'] = df_libros['Libro'].apply(limpieza_top)

In [ ]:
df_libros.drop(df_libros[df_libros['es_ruido'] == True].index, inplace=True)

In [ ]:
df_libros = df_libros.drop('es_ruido', axis=1) # dropeo columna auxiliar.

In [ ]:
df_libros.reset_index(drop=True, inplace=True) # reseteo indices, ya que eliminé las filas que no son libros.

In [ ]:
df_libros = df_libros.drop_duplicates() # elimino duplicados si los hubiera

In [ ]:
df_libros_filtered: pd.DataFrame = df_libros.copy()
df_libros_filtered['Libro'] = df_libros['Libro'].str.replace(r'\s*\(\d+\)$', '', regex=True)


### df_libros: segundo enfoque

Hacer scrapping sobre la pagina para obterner el link de referencia de cada libro.

Por ejermplo: 'Romeo and Juliet by William Shakespeare (2410)' hace referencia a '/ebook/1513'. De esa forma se puede ingresar a https://www.gutenberg.org/ebooks/1513 y obtener así una breve descripcion del libro (necesaria para la recomendacion final)


In [ ]:
url: str = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'

In [ ]:
"""
Esta celda se encarda de hacer web scrapping sobre la lista con todos los libros.
De ella, se obtiene tanto el título del libro como su link link de referencia.

La información es guardada sobre el dataframe 'books_df'

"""

response: requests.models = requests.get(url)
boup: BeautifulSoup = BeautifulSoup(response.text, 'html.parser')
libro_div = soup.find('div', class_='page_content')

books: list[dict[str,str]] = []

for a in libro_div.find_all('a', href=True):
    title = a.get_text(strip=True)
    link = a['href']
    books.append({'title': title, 'link': link})

books_df: pd.DataFrame = pd.DataFrame(books, columns=['title', 'link'])

In [ ]:
# Se elimina todo elemento fuera del Top 1000
books_df = books_df.iloc[7:1007].reset_index(drop=True)

In [ ]:
books_df.head()

,title,link
0,"Frankenstein; Or, The Modern Prometheus by Mar...",/ebooks/84
1,The Legend of Sleepy Hollow by Washington Irvi...,/ebooks/41
2,Pride and Prejudice by Jane Austen (2536),/ebooks/1342
3,"Moby Dick; Or, The Whale by Herman Melville (2...",/ebooks/2701
4,Romeo and Juliet by William Shakespeare (2132),/ebooks/1513


Una vez obtenido el DataFrame con los libros pertenecientes al Top 1000, nos resulta de interés tener una breve descripción sobre cada libro.  

Para ello, se itera sobre el link de referencia de cada libro.

Analizar si es necesario aplicar técnicas para limpiar la descripcion de cada libro

In [ ]:
# ANALIZAR GUARDAR ESTE DF PARA NO VOLVER A CORRER EL CODIGO, TARDA COMO 3 MINS


for index, link in enumerate(books_df['link']):

  url = f"https://www.gutenberg.org{link}"
  # url= 'https://www.gutenberg.org/ebooks/348'

  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  # Buscar el div con id 'bibrec'
  bio_div = soup.find('div', id='bibrec')

  # Buscar el tr que contiene el th con el texto 'Summary'
  summary_tr = bio_div.find('th', string='Summary')

  # Si encontró el 'th' correcto, obtener el contenido del 'td' siguiente
  if summary_tr:
      summary_text = summary_tr.find_parent('tr').find('td').get_text(strip=True)
  else:
      summary_text = 'Descripción no encontrada'

  books_df.at[index, 'summary'] = summary_text


In [ ]:
books_df.sample(20)

,title,link,summary
336,"Pamela, or Virtue Rewarded by Samuel Richardso...",/ebooks/6124,"""Pamela, or Virtue Rewarded"" by Samuel Richard..."
467,"Human, All Too Human: A Book for Free Spirits ...",/ebooks/38145,"""Human, All Too Human: A Book for Free Spirits..."
176,Vanity Fair by William Makepeace Thackeray (197),/ebooks/599,"""Vanity Fair"" by William Makepeace Thackeray i..."
772,A Dictionary of Cebuano Visayan by John U. Wol...,/ebooks/40074,"""A Dictionary of Cebuano Visayan"" by John U. W..."
169,Childe Harold's Pilgrimage by Baron George Gor...,/ebooks/5131,"""Childe Harold's Pilgrimage"" by Lord Byron is ..."
777,"Plutarch's Lives, Volume 1 (of 4) by Plutarch ...",/ebooks/14033,"""Plutarch's Lives, Volume 1"" by Plutarch is a ..."
257,The Idiot by Fyodor Dostoyevsky (140),/ebooks/2638,"""The Idiot"" by Fyodor Dostoyevsky is a novel w..."
15,The Great Gatsby by F. Scott Fitzgerald (1366),/ebooks/64317,"""The Great Gatsby"" by F. Scott Fitzgerald is a..."
144,"Hamlet, Prince of Denmark by William Shakespea...",/ebooks/1524,"""Hamlet, Prince of Denmark"" by William Shakesp..."
878,The Phantom-Wooer by Thomas Lovell Beddoes (52),/ebooks/21170,Descripción no encontrada


In [ ]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1000 non-null   object
 1   link     1000 non-null   object
 2   summary  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [ ]:
(books_df['summary'] == 'Descripción no encontrada').sum()

183

In [ ]:
df_books_filtered : pd.DataFrame = books_df.copy()
df_books_filtered['title'] = df_books_filtered['title'].str.replace(r'\s*\(\d+\)$', '', regex=True)

In [45]:
df_books_filtered.to_csv('/content/books.csv')

## Modelado

In [42]:
# Descargamos los stopwords que necesitaremos luego
nltk.download('stopwords')
from nltk.corpus import stopwords

# Obtenemos las stopwords para español
spanish_stop_words = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
